# Creating Predictions
If you have some HSI images and want to get predictions from one of our pretrained models (or your own models), then we covered you in this notebook. Predictions can be computed based on a folder with HSI images and the `htc inference` command.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from htc import Config, LabelMapping, decompress_file, settings

In [2]:
data_dir = settings.data_dirs["HeiPorSPECTRAL"]
output_dir = settings.results_dir / "predictions"

> Note: If you want to use your own data and it does not fit into the default structure (e.g. because you have non-Tivita images), they you need to write your [own DataPath class](./CustomDataPath.md) first.

In [3]:
!htc inference --input-dir $data_dir/subjects/P086/2021_04_15_09_22_02 --output-dir $output_dir --model image --run-folder 2022-02-03_22-58-44_generated_default_model_comparison
assert _exit_code == 0, "Inference was not successful"  # noqa: F821

[INFO][htc.no_duplicates] Found pretrained run in the local hub  HTCModel.py:180
dir at                                                                          
/home/j562r/.cache/torch/hub/htc_checkpoints/image/2022-02-03_22                
-58-44_generated_default_model_comparison                                       
Dataloader ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:01


This command searches for all HSI images in the given input directory, computes a prediction using the specified trained model (will also be downloaded if not available) and stores the result in the given output directory. You can use any of the pretrained models here.

In this example case, there is only one output image:

In [4]:
sorted(output_dir.rglob("*"))

[PosixPath('/mnt/ssd_8tb/htc/results_test/predictions/2022-02-03_22-58-44_generated_default_model_comparison'),
 PosixPath('/mnt/ssd_8tb/htc/results_test/predictions/2022-02-03_22-58-44_generated_default_model_comparison/0202-00118#2021_04_15_09_22_02.blosc'),
 PosixPath('/mnt/ssd_8tb/htc/results_test/predictions/2022-02-03_22-58-44_generated_default_model_comparison/0202-00118#2021_04_15_09_22_02.html'),
 PosixPath('/mnt/ssd_8tb/htc/results_test/predictions/2022-02-03_22-58-44_generated_default_model_comparison/config.json')]

Per default, this includes the predicted labels (stored in the blosc file), a visualization of the prediction (HTML file) and the config which was used for computing the predictions (which again is based on the config of the trained model). You can open the HTML file directly with any browser. The labels can be read with the `decompress_file()` function:

In [5]:
labels = decompress_file(sorted(output_dir.rglob("*.blosc"))[0])
labels

array([[ 0,  0,  0, ..., 12, 12,  0],
       [ 0,  0,  0, ..., 12, 12,  0],
       [ 0,  0,  0, ..., 12, 12, 12],
       ...,
       [ 5,  5,  5, ...,  4,  4,  4],
       [ 5,  5,  5, ...,  4,  4,  4],
       [ 5,  5,  5, ...,  4,  4,  4]])

The config can, for example, be used to recover the original label names:

In [6]:
config = Config(sorted(output_dir.rglob("config.json"))[0])
mapping = LabelMapping.from_config(config)

for l, c in zip(*np.unique(labels, return_counts=True)):
    print(f"{mapping.index_to_name(l)}: {c} pixels")

background: 55531 pixels
liver: 26718 pixels
colon: 97583 pixels
small_bowel: 1090 pixels
stomach: 37645 pixels
spleen: 42066 pixels
omentum: 9688 pixels
peritoneum: 20739 pixels
skin: 5816 pixels
fat_subcutaneous: 10324 pixels


> Note: if you need the softmax values instead of the label indices of the prediction, add the `--predictions-type softmax` switch to `htc inference`. However, be aware that this requires much more disk space (around 17 MiB instead of 2 KiB per image)